In [53]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import os
from pathlib import Path
import matplotlib.pyplot as plt
import csv

In [54]:
#데이터셋
DATASET_NAME = 'kfood'
DRIVE_PATH = Path(os.getcwd())
DATASET_PATH = DRIVE_PATH / DATASET_NAME
filepath = DATASET_PATH
print(filepath.exists())

True


In [115]:
#class_label 매칭 딕셔너리로 저장
labels = None
classes = None
with open(filepath / 'class_label.csv','r') as f:
    w = csv.reader(f)
    classes = w.__next__()
    labels = w.__next__()
#print(len(classes), len(labels), classes, labels)
class_to_label = {}
for _class, _label in zip(classes, labels):
    class_to_label[_class] = int(_label)

print(class_to_label.items())

dict_items([('떡꼬치', 0), ('편육', 1), ('후라이드치킨', 2), ('콩자반', 3), ('젓갈', 4), ('양념치킨', 5), ('과메기', 6), ('피자', 7), ('육회', 8), ('물회', 9), ('알밥', 10), ('누룽지', 11), ('김치볶음밥', 12), ('김밥', 13), ('비빔밥', 14), ('새우볶음밥', 15), ('유부초밥', 16), ('잡곡밥', 17), ('주먹밥', 18), ('회무침', 19), ('꽈리고추무침', 20), ('도라지무침', 21), ('콩나물무침', 22), ('홍어무침', 23), ('잡채', 24), ('도토리묵', 25), ('만두', 26), ('더덕구이', 27), ('떡갈비', 28), ('갈비구이', 29), ('훈제오리', 30), ('장어구이', 31), ('삼겹살', 32), ('불고기', 33), ('황태구이', 34), ('조기구이', 35), ('조개구이', 36), ('갈치구이', 37), ('고등어구이', 38), ('곱창구이', 39), ('닭갈비', 40), ('미역줄기볶음', 41), ('고사리나물', 42), ('가지볶음', 43), ('애호박볶음', 44), ('숙주나물', 45), ('시금치나물', 46), ('갈비탕', 47), ('매운탕', 48), ('감자탕', 49), ('추어탕', 50), ('곰탕_설렁탕', 51), ('삼계탕', 52), ('보쌈', 53), ('육개장', 54), ('북엇국', 55), ('콩나물국', 56), ('무국', 57), ('계란국', 58), ('미역국', 59), ('떡국_만두국', 60), ('시래기국', 61), ('간장게장', 62), ('양념게장', 63), ('물냉면', 64), ('수제비', 65), ('콩국수', 66), ('막국수', 67), ('열무국수', 68), ('비빔냉면', 69), ('짜장면', 70), ('쫄면', 71), ('칼국수', 72), ('잔치국수', 

In [116]:
#데이터셋의 이미지 경로 및 레이블 저장
from glob import glob
image_paths = sorted(glob("kfood/*/*/*"))
image_paths = [image_path for image_path in image_paths if image_path.split("/")[-1].split(".")[-1].lower() in ("png", "jpg", "jpeg")]
#labels = [class_to_label[Path(image_path).parent.stem] for image_path in image_paths]
print(len(image_paths))#, len(labels))


150507


In [117]:
def get_image_crop_points(filepath):
    crops = {}
    properties = filepath / "crop_area.properties"
    with open(properties, 'r') as p:
        for row in p:
            name, crop = row.replace("\n", "").replace(" ", "").split("=")
            if name != "" and crop != "":
                crop = crop.split(",")
                if len(crop) >= 4:
                    crop = [int(crop[1]), int(crop[0]), int(crop[3]), int(crop[2])]
                    crops[name] = crop
                elif len(crop) == 2:
                    crop = [0, 0, int(crop[1]), int(crop[0])]
                    crops[name] = crop
            
    return crops

def parse_and_crop_image_add_label(filepath):

    image = tf.io.read_file(filepath) # 이미지 파일 읽기

    #format decoding
    image_format = Path(filepath).suffix.lower()
    if image_format == ".jpeg":
        image = tf.image.decode_jpeg(image, channels=3) # JPEG-encoded -> uint8 tensor (RGB format)
    elif image_format == '.png':
        image = tf.image.decode_png(image, channels=3)
    else:
        image = tf.image.decode_image(image, channels=3)

    #crop
    image_name = Path(filepath).name
    if crop_points.get(image_name):
        image = tf.image.crop_to_bounding_box(image, *crop_points[image_name])
    
    #labeling
    class_name = Path(filepath).parent.name
    label = class_to_label.get(class_name)
    if label == None:
        label = 0
    return image, int(label)

In [118]:
#print(Path(image_paths[0]).parent.stem)
#print(class_to_label.get(Path(image_paths[0]).parent.stem))
#print(image_paths[0], parse_and_crop_image_add_label(image_paths[0]))

갈비구이
29
kfood/구이/갈비구이/Img_000_0000.jpg (<tf.Tensor: shape=(600, 900, 3), dtype=uint8, numpy=
array([[[192, 213, 218],
        [192, 213, 218],
        [192, 213, 218],
        ...,
        [209, 229, 236],
        [209, 229, 236],
        [209, 229, 236]],

       [[192, 213, 218],
        [192, 213, 218],
        [192, 213, 218],
        ...,
        [209, 229, 236],
        [209, 229, 236],
        [209, 229, 236]],

       [[192, 213, 218],
        [192, 213, 218],
        [192, 213, 218],
        ...,
        [210, 230, 237],
        [210, 230, 237],
        [210, 230, 237]],

       ...,

       [[133, 149, 149],
        [132, 148, 148],
        [131, 147, 147],
        ...,
        [213, 232, 239],
        [213, 232, 239],
        [213, 232, 239]],

       [[134, 150, 150],
        [133, 149, 149],
        [132, 148, 148],
        ...,
        [213, 231, 241],
        [213, 231, 241],
        [213, 231, 241]],

       [[135, 151, 151],
        [134, 150, 150],
        [133, 149, 

In [119]:
def make_kfood_dataset(filepaths, n_read_threads=5, shuffle_buffer_size=None, n_parse_threads=5, batch_size=32, cache=False):

    print(1)
    filenames_dataset = tf.data.Dataset.from_tensor_slices(filepaths)
    print(1)
    dataset = filenames_dataset.map(parse_and_crop_image_add_label, num_parallel_calls=n_parse_threads)
    print(1)
    if cache:
        dataset = dataset.cache()
    if shuffle_buffer_size:
        dataset = dataset.shuffle(shuffle_buffer_size)
    dataset = dataset.batch(batch_size)
    return dataset.prefetch(1)

In [120]:
#crop 지점 정보 빼오기
crop_points = {}
class_list = list(filepath.glob("*/*"))
print(len(class_list))
class_list = [class_name for class_name in class_list if class_name.is_dir()]
print(len(class_list))
for class_name in class_list:
    crop_points.update(get_image_crop_points(class_name))

300
150


In [121]:
#데이터셋 생성
dataset = make_kfood_dataset(image_paths, shuffle_buffer_size=10000)

1
1


TypeError: in user code:

    /var/folders/ty/sj9j6sl158j5bv97y8pxd76c0000gn/T/ipykernel_97974/1612623417.py:23 parse_and_crop_image_add_label  *
        image_format = Path(filepath).suffix.lower()
    /Users/hwijin/miniforge3/envs/tf25/lib/python3.8/pathlib.py:1042 __new__  **
        self = cls._from_parts(args, init=False)
    /Users/hwijin/miniforge3/envs/tf25/lib/python3.8/pathlib.py:683 _from_parts
        drv, root, parts = self._parse_args(args)
    /Users/hwijin/miniforge3/envs/tf25/lib/python3.8/pathlib.py:667 _parse_args
        a = os.fspath(a)

    TypeError: expected str, bytes or os.PathLike object, not Tensor


In [76]:
crop_points = {}
class_list = list(filepath.glob("*/*"))
print(len(class_list))
class_list = [class_name for class_name in class_list if class_name.is_dir()]
print(len(class_list))
for class_name in class_list:
    crop_points.update(get_image_crop_points(class_name))

300
150


In [77]:
#{ image_filename : crop_point, ... }
len(crop_points.keys())

51660

In [95]:
from PIL import Image
tf.shape(parse_and_crop_image(image_paths[9]))

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([573, 764,   3], dtype=int32)>

In [30]:
filenames_dataset = tf.data.Dataset.from_tensor_slices(image_paths)
images_ds = filenames_dataset.map(parse_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)


tf.Tensor([], shape=(0,), dtype=int32)
tf.Tensor([400 534   3], shape=(3,), dtype=int32)


In [7]:
feature_descriptions = {
    "image": tf.io.FixedLenFeature([], tf.string, default_value=""),
    "label": tf.io.FixedLenFeature([], tf.int64, default_value=0)
}
for tfrecord in dataset.take(1):
    example = tf.io.parse_single_example(tfrecord, feature_descriptions)

2021-12-24 13:16:07.492231: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-12-24 13:16:07.492404: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


KeyboardInterrupt: 